In [3]:
import os
import glob
import inro.modeller as m
mm = m.Modeller()    
NAMESPACE = "inro.emme.data.matrix.change_matrix_properties"
change_matrix = m.Modeller().tool(NAMESPACE)
NAMESPACE = "tmg.input_output.import_binary_matrix"
import_matrices = mm.tool(NAMESPACE)
eb = m.Modeller().emmebank

In [4]:
scenario_2011 = 101
scenario_2016 = 109
scenario_2041 = 114

ntwk_2011 = eb.scenario(scenario_2011).get_network()
ntwk_2016 = eb.scenario(scenario_2016).get_network()
ntwk_2041 = eb.scenario(scenario_2041).get_network()

In [21]:
ntwk = ntwk_2011

sov_total = 0.0
all_total = 0.0
for link in ntwk.links():
    #ignore centroid connectors
    if not link.i_node.is_centroid and not link.j_node.is_centroid:
        #ignore non-auto
        if ntwk.mode('c') in link.modes:
            all_total += link.data3*link.num_lanes*link.length
            if ntwk.mode('h')not in link.modes or ntwk.mode('j') in link.modes :
                sov_total += link.data3*link.num_lanes*link.length
            
print "{:,.2f}".format(sov_total)
print '{:,.2f}'.format(all_total)

59,779,648.52
59,936,687.10


In [33]:
### IMPORT ALL MATRICES IN SUBFOLDERS ###
def import_matrices_from_folder(matrix_folder, matrix_list,scenario):

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(".mdf"):
                name = matrix_file[:-4]
                if name in matrix_list:
                    matrix_number = matrix_list[name]
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name )
                    change_matrix(matrix = "mf" + str(matrix_number),
                        matrix_name = name[:35],
                      matrix_description = name)
                    print "Imported matrix: " + matrix_file + " to: " + str(matrix_number)
    print "Finshed Importing Matrices"

In [69]:
#import all matrices from run output folder

#Delete all matrices
for matrix in eb.matrices():
    eb.delete_matrix(matrix.id)

matrix_folder = u"\\\\tore-infs01\\PWExternal\\451010\\281829_GGH\\GGHM Runs\\2041\\MTP_304\\"
#scenario compatible with truck matrices
truck_scen = 103
scen_num = 114

#Transit Travel Time matrices - find minimum time for the sum of times for each mode
matrix_list={'skim.peak.transit.local_bus.ivt.bus':86,
             'skim.peak.transit.local_bus.ivt.streetcar':85,
             'skim.peak.transit.rapid_bus.ivt.bus':12, 
             'skim.peak.transit.rapid_bus.ivt.streetcar':15,
             'skim.peak.transit.rapid_bus.ivt.rapid_bus':17,
             'skim.peak.transit.go_bus.ivt.bus':47,
             'skim.peak.transit.go_bus.ivt.streetcar':50,
             'skim.peak.transit.go_bus.ivt.rapid_bus':52,
             'skim.peak.transit.go_bus.ivt.go_bus':55,
             'skim.peak.transit.go_bus.ivt.premium_bus':57,
             'skim.peak.transit.ztsa.ivt.bus':93,
             'skim.peak.transit.ztsa.ivt.streetcar':96,
             'skim.peak.transit.ztsa.ivt.rapid_bus':98,
             'skim.peak.transit.ztsa.ivt.go_bus':21,
             'skim.peak.transit.ztsa.ivt.premium_bus':23,
             'skim.peak.transit.subway.ivt':24,
             'skim.peak.transit.train.ivt':25
            }

import_matrices_from_folder(matrix_folder, matrix_list,scen_num)

#Calculate overall transit time matrix
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")


def remove_zeros_spec(matrix_id):
    spec = {
    "type": "MATRIX_CALCULATION",
    "result": matrix_id,
    "expression": "1000",
    "constraint": {
        "by_value": {
            "od_values": matrix_id,
            "interval_min": 0,
            "interval_max": 0,
            "condition": "INCLUDE"
            }
        }
    }
    return spec

#Local Bus
local_bus_matrix = "mf1"
create_matrix(matrix_id = local_bus_matrix,
              matrix_name="skim.peak.transit.local_bus.ivt",
              matrix_description = "skim.peak.transit.local_bus.ivt")
spec = {
    "type": "MATRIX_CALCULATION",
    "result": local_bus_matrix,
    "expression": "(mf86+mf85)"
    }
matrix_calculator(spec)
matrix_calculator(remove_zeros_spec(local_bus_matrix))

#Rapid Bus
rapid_bus_matrix = "mf2"
create_matrix(matrix_id = rapid_bus_matrix,
              matrix_name="skim.peak.transit.rapid_bus.ivt",
              matrix_description = "skim.peak.transit.rapid_bus.ivt")
spec = {
    "type": "MATRIX_CALCULATION",
    "result": rapid_bus_matrix,
    "expression": "(mf12+mf15+mf17)"
    }
matrix_calculator(spec)
matrix_calculator(remove_zeros_spec(rapid_bus_matrix))

#GO Bus
go_bus_matrix = "mf3"
create_matrix(matrix_id = go_bus_matrix,
              matrix_name="skim.peak.transit.go_bus.ivt",
              matrix_description = "skim.peak.transit.go_bus.ivt")
spec = {
    "type": "MATRIX_CALCULATION",
    "result": go_bus_matrix,
    "expression": "(mf47+mf50+mf52+mf55+mf57)"
    }
matrix_calculator(spec)
matrix_calculator(remove_zeros_spec(go_bus_matrix))

#Subway
subway_matrix = "mf4"
create_matrix(matrix_id = subway_matrix,
              matrix_name="skim.peak.transit.subway.ivt",
              matrix_description = "skim.peak.transit.subway.ivt")
spec = {
    "type": "MATRIX_CALCULATION",
    "result": subway_matrix,
    "expression": "(mf93+mf96+mf98+mf21+mf23+mf24)"
    }
matrix_calculator(spec)
matrix_calculator(remove_zeros_spec(subway_matrix))

#Train
train_matrix = "mf5"
create_matrix(matrix_id = train_matrix,
              matrix_name="skim.peak.transit.train.ivt",
              matrix_description = "skim.peak.transit.train.ivt")
spec = {
    "type": "MATRIX_CALCULATION",
    "result": train_matrix,
    "expression": "(mf93+mf96+mf98+mf21+mf23+mf25)"
    }
matrix_calculator(spec)
matrix_calculator(remove_zeros_spec(train_matrix))

#find minimum travel time
min_time_matrix = "mf6"
create_matrix(matrix_id = min_time_matrix,
              matrix_name="skim.peak.transit.ivt",
              matrix_description = "skim.peak.nongotrain.transit.ivt")
spec = {
    "type": "MATRIX_CALCULATION",
    "result": min_time_matrix,
    "expression": "mf1.min.mf2.min.mf3.min.mf4",
}
matrix_calculator(spec)

spec = {
    "type": "MATRIX_CALCULATION",
    "result": min_time_matrix,
    "expression": "0",
    "constraint": {
    "by_value": {
            "od_values": min_time_matrix,
            "interval_min": 1000,
            "interval_max": 1000,
            "condition": "INCLUDE"
        }
    }
}
matrix_calculator(spec)

spec = {
    "type": "MATRIX_CALCULATION",
    "result": train_matrix,
    "expression": "0",
    "constraint": {
    "by_value": {
            "od_values":  train_matrix,
            "interval_min": 1000,
            "interval_max": 1000,
            "condition": "INCLUDE"
        }
    }
}
matrix_calculator(spec)

for matrix in eb.matrices():
    if matrix.id != min_time_matrix:
        eb.delete_matrix(matrix.id)

print "Finished Calculating Transit Travel Times"
#Truck Peak Demand Matrices - find maximum demand for: am vs pm, matrix vs transpose

matrix_list={'lt_truck_ampkhr': 84,
            'lt_truck_mdpkhr': 86,
            'mt_truck_ampkhr': 89,
            'mt_truck_mdpkhr': 91,
            'ht_truck_ampkhr': 94,
            'ht_truck_mdpkhr': 96}

import_matrices_from_folder(matrix_folder, matrix_list,truck_scen)

#Other matrices - can export without any other processing

matrix_list={'skim.am_peak.auto.sov.dist':27,
             'skim.midday.auto.sov.time':21,
             'skim.am_peak.auto.sov.time':12,
             'skim.am_peak.auto.hov2.time':17,
             'skim.static.bike.time':7,
             'skim.am_peak.auto.sovt.toll_cost':35,
             'skim.am_peak.auto.hov2t.toll_cost':50,
             'skim.offpeak.transit.go_bus.fares':31}

import_matrices_from_folder(matrix_folder, matrix_list,scen_num)

#Transit Access time - sum matrices
matrix_list={'skim.peak.transit.local_bus.total_wait': 71,
             'skim.peak.transit.local_bus.access_walk': 76,
             'skim.peak.transit.local_bus.egress_walk': 81}

import_matrices_from_folder(matrix_folder, matrix_list,scen_num)

#demand matrices
matrix_list = {'trips_peak_sov' : 40,
               'trips_peak_hov2' : 41,
               'trips_peak_hov3pl' : 42,
               'trips_offpeak_sov' : 43,
               'trips_offpeak_hov2' : 44,
               'trips_offpeak_hov3pl' : 45,
               'trips_peak_bike' : 46,
               'trips_peak_walk': 47,
               'trips_offpeak_bike': 48,
               'trips_offpeak_walk': 49,
               'trips_transit_peak': 51,
               'trips_transit_offpeak': 52
               }
import_matrices_from_folder(matrix_folder, matrix_list,scen_num)

Imported matrix: skim.peak.transit.go_bus.ivt.bus.mdf to: 47
Imported matrix: skim.peak.transit.go_bus.ivt.go_bus.mdf to: 55
Imported matrix: skim.peak.transit.go_bus.ivt.premium_bus.mdf to: 57
Imported matrix: skim.peak.transit.go_bus.ivt.rapid_bus.mdf to: 52
Imported matrix: skim.peak.transit.go_bus.ivt.streetcar.mdf to: 50
Imported matrix: skim.peak.transit.local_bus.ivt.bus.mdf to: 86
Imported matrix: skim.peak.transit.local_bus.ivt.streetcar.mdf to: 85
Imported matrix: skim.peak.transit.rapid_bus.ivt.bus.mdf to: 12
Imported matrix: skim.peak.transit.rapid_bus.ivt.rapid_bus.mdf to: 17
Imported matrix: skim.peak.transit.rapid_bus.ivt.streetcar.mdf to: 15
Imported matrix: skim.peak.transit.subway.ivt.mdf to: 24
Imported matrix: skim.peak.transit.train.ivt.mdf to: 25
Imported matrix: skim.peak.transit.ztsa.ivt.bus.mdf to: 93
Imported matrix: skim.peak.transit.ztsa.ivt.go_bus.mdf to: 21
Imported matrix: skim.peak.transit.ztsa.ivt.premium_bus.mdf to: 23
Imported matrix: skim.peak.transi

In [70]:
#Calculate demand matrices

new_matrix = "mf32"
create_matrix(matrix_id = new_matrix,
              matrix_name="trips.peak.hov",
            matrix_description = "trips.peak.hov")   
spec = {
    "type": "MATRIX_CALCULATION",
    "result": new_matrix,
    "expression": "mf41+mf42"
}
matrix_calculator(spec)

new_matrix = "mf33"
create_matrix(matrix_id = new_matrix,
              matrix_name="trips.offpeak.hov",
            matrix_description = "trips.offpeak.hov")   
spec = {
    "type": "MATRIX_CALCULATION",
    "result": new_matrix,
    "expression": "mf44+mf45"
}
matrix_calculator(spec)

new_matrix = "mf34"
create_matrix(matrix_id = new_matrix,
              matrix_name="trips.peak.active",
            matrix_description = "trips.peak.active")   
spec = {
    "type": "MATRIX_CALCULATION",
    "result": new_matrix,
    "expression": "mf46+mf47"
}
matrix_calculator(spec)

new_matrix = "mf36"
create_matrix(matrix_id = new_matrix,
              matrix_name="trips.offpeak.active",
            matrix_description = "trips.offpeak.active")   
spec = {
    "type": "MATRIX_CALCULATION",
    "result": new_matrix,
    "expression": "mf48+mf49"
}
matrix_calculator(spec)

for matrix in [41,42,44,45,46,47,48,49]:
    eb.delete_matrix("mf" + str(matrix))
print "Finished calculating demand"

Finished calculating bi-directional peak travel time


In [71]:
#Calculate overall Truck matrices for: light, medium, heavy
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")

#Light Trucks
new_matrix = "mf85"
create_matrix(matrix_id = new_matrix,
              matrix_name="trips.peak.truck.light",
            matrix_description = "trips.peak.truck.light")
spec = {
    "type": "MATRIX_CALCULATION",
    "result": new_matrix,
    "expression": "(mf84.max.mf84').max.(mf86.max.mf86')"
}
matrix_calculator(spec)
eb.delete_matrix("mf84")
eb.delete_matrix("mf86")

#Medium Trucks
new_matrix = "mf90"
create_matrix(matrix_id = new_matrix,
              matrix_name="trips.peak.truck.medium",
            matrix_description = "trips.peak.truck.medium")
spec = {
    "type": "MATRIX_CALCULATION",
    "result": new_matrix,
    "expression": "(mf89.max.mf89').max.(mf91.max.mf91')"
}
matrix_calculator(spec)
eb.delete_matrix("mf89")
eb.delete_matrix("mf91")

#Heavy Trucks
new_matrix = "mf95"
create_matrix(matrix_id = new_matrix,
              matrix_name="trips.peak.truck.heavy",
            matrix_description = "trips.peak.truck.heavy")
spec = {
    "type": "MATRIX_CALCULATION",
    "result": new_matrix,
    "expression": "(mf94.max.mf94').max.(mf96.max.mf96')"
}
matrix_calculator(spec)
eb.delete_matrix("mf94")
eb.delete_matrix("mf96")

print "Finished creating truck matrices"


Finished creating truck matrices


In [72]:
#Calculate bi-drectional peak travel time
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")

new_matrix = "mf1"
create_matrix(matrix_id = new_matrix,
              matrix_name="temp.matrix",
            matrix_description = "temp.matrix")

for matrix in eb.matrices():
    if matrix.id in ["mf12","mf17","mf21"]:        
        spec = {
            "type": "MATRIX_CALCULATION",
            "result": new_matrix,
            "expression": "(" + matrix.id + ".max." + matrix.id + "')"
        }
        matrix_calculator(spec)
        spec = {
            "type": "MATRIX_CALCULATION",
            "result": matrix.id,
            "expression": new_matrix
        }
        matrix_calculator(spec)
eb.delete_matrix(new_matrix)
print "Finished calculating bi-directional peak travel time"

Finished calculating bi-directional peak travel time


In [73]:
#Calculate transit access time matrix
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")


new_matrix = "mf1"
if not eb.matrix(new_matrix):
    create_matrix(matrix_id = new_matrix,
              matrix_name="skim.peak.transit.access",
            matrix_description = "skim.peak.transit.access")
spec = {
    "type": "MATRIX_CALCULATION",
    "result": new_matrix,
    "expression": "mf71+mf76+mf81"
}
matrix_calculator(spec)
to_delete = ["mf71", "mf76", "mf81"]
for matrix in to_delete:
    eb.delete_matrix(matrix)
print "Finished Calculating Transit Access Times"

Finished Calculating Transit Access Times


In [74]:
output_map = {
             #matrices outputted directly 
             "mf27":'skim.am_peak.auto.sov.dist',
             "mf21":'skim.midday.auto.sov.time',
             "mf12":'skim.am_peak.auto.sov.time',
             "mf17":'skim.am_peak.auto.hov2.time',
             "mf7":'skim.static.bike.time',
             "mf35":'skim.am_peak.auto.sovt.toll_cost',
             "mf50":'skim.am_peak.auto.hov2t.toll_cost',
             "mf31":'skim.offpeak.transit.go_bus.fares',
             #matrices resulting from calculations
             "mf85":'trips.peak.truck.light',
             "mf90":'trips.peak.truck.medium',
             "mf95":'trips.peak.truck.heavy',
             "mf1": 'skim.peak.transit.access',
             "mf6": 'skim.peak.nongotrain.transit.ivt',
             "mf32": 'trips.peak.hov',
    "mf33":'trips.offpeak.hov',
    "mf34":'trips.peak.active',
    "mf36":'trips.offpeak.active',
    "mf40":'trips.peak.sov',
    "mf43":'trips.offpeak.sov',
    "mf51":'trips.peak.transit',
    "mf52":'trips.offpeak.transit'
}

In [ ]:
#EXPORT ALL MATRICES INTO EXCEL FILES 

save_path = "C:\Users\mnasterska\Documents\Projects\GGH\SketchInputs\ZoneMatrices\\"

NAMESPACE = "inro.emme.data.matrix.export_matrix_to_csv"
export_matrices = m.Modeller().tool(NAMESPACE)
base_scenario = mm.scenario
for matrix in eb.matrices():
    if matrix.id in output_map:
        export_matrices(matrices=[matrix.id],
                        export_path = save_path)
        print "Exported matrix " + matrix.id
print "Finished exporting matrices."

Exported matrix mf1
Exported matrix mf6